In [75]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

import app.env

YOUTUBE_API_KEY = app.env.YOUTUBE_API_KEY

In [76]:
from dataclasses import dataclass
from datetime import datetime
from typing import Generator, Optional, Dict, Any
import json

@dataclass
class VideoInfo:
    video_id: str
    title: str
    channel: str
    published_at: datetime
    description: str
    channel_id: str
    view_count: Optional[int] = None
    duration: Optional[str] = None

@dataclass
class CommentInfo:
    comment_id: str
    video_id: str
    text: str
    author: str
    like_count: int
    published_at: datetime
    updated_at: datetime
    reply_count: int
    channel_id: str
    author_channel_id: str

In [ ]:
from datetime import timedelta
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import logging

from app.schemas import YoutubeVideoInfoSchema

logger = logging.getLogger(__name__)

def search_youtube_videos(
    query: str,
    max_results: int = 50
) -> Generator[YoutubeVideoInfoSchema, None, None]:
    """
    Search for YouTube videos by keyword using a generator.

    Yields:
        VideoInfo objects one at a time
    """
    if not YOUTUBE_API_KEY:
        raise ValueError("YouTube API key not configured")

    try:
        youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)

        # Handle pagination
        next_page_token = None
        videos_yielded = 0

        while videos_yielded < max_results:
            if max_results > 50:
                logger.warning("YouTube API limits batch size to 50 results.")
            batch_size = min(50, max_results - videos_yielded)  # YouTube API max is 50

            delta_day = datetime.now() - timedelta(days=2)

            request = youtube.search().list(
                q=query,
                part="id,snippet",
                maxResults=batch_size,
                type="video",
                pageToken=next_page_token,
                order="viewCount",
                publishedAfter=delta_day.isoformat() + 'Z',
                relevanceLanguage="en",
                regionCode="US",
            )
            response = request.execute()

            # Process each video in the response
            for item in response.get("items", []):
                if videos_yielded >= max_results:
                    break

                try:
                    video_info = _parse_video_item(item)
                    yield video_info
                    videos_yielded += 1
                except Exception as e:
                    print(f"Error parsing video item: {e}")
                    continue

            # Check if there are more pages
            next_page_token = response.get("nextPageToken")
            if not next_page_token or videos_yielded >= max_results:
                break

    except HttpError as e:
        print(f"YouTube API error: {e}")
        return
    except Exception as e:
        print(f"Error: {e}")
        return

def _parse_video_item(item: Dict[str, Any]) -> VideoInfo:
    """Parse a YouTube API video item into a VideoInfo object."""
    snippet = item["snippet"]

    return VideoInfo(
        video_id=item["id"]["videoId"],
        title=snippet["title"],
        channel=snippet["channelTitle"],
        published_at=datetime.fromisoformat(
            snippet["publishedAt"].replace('Z', '+00:00')
        ),
        description=snippet["description"],
        channel_id=snippet["channelId"]
    )

In [78]:
from typing import Callable, Iterable, Generator, TypeVar

T = TypeVar('T')

def compose_filters(*filters: Callable[[Iterable[T]], Iterable[T]]) -> Callable[[Iterable[T]], Iterable[T]]:
    """
    Compose multiple filter functions for any type.
    Each filter should accept and return an iterable of T.
    """
    def composed_filter(items: Iterable[T]) -> Iterable[T]:
        current = items
        for filter_func in filters:
            current = filter_func(current)
        return current
    return composed_filter

In [79]:
import re
from collections import Counter
from typing import Set, List, Dict, Tuple
from langdetect import detect, LangDetectException
from emoji import emoji_count
from collections import Counter

# Configuration constants
SPAM_PATTERNS = [
    r'(?i)\b(subscribe|like|follow)\b.*\b(back|sub4sub|f4f)\b',
    r'(?i)\b(check.*out|visit).*\b(channel|profile|link)\b',
    r'(?i)\b(make.*money|earn.*\$|click.*here|amazing.*opportunity)\b',
    r'(?i)\b(bot|automated|script)\b',
    r'^\s*[!@#$%^&*()_+=\[\]{}|;:,.<>?]*\s*$',  # Only special characters
    r'(?i)\b(first|early|notification.*squad)\b',
    r'https?://\S+',  # URLs
]

def create_comment_fingerprint(text: str) -> str:
    """Create a normalized fingerprint of the comment."""
    # Remove special chars, convert to lowercase, remove extra spaces
    normalized = re.sub(r'[^\w\s]', '', text.lower())
    normalized = re.sub(r'\s+', ' ', normalized).strip()
    # Sort words to catch reordered spam
    words = sorted(normalized.split())
    return ' '.join(words)

def check_spam_patterns(text: str) -> float:
    """Check for known spam patterns."""
    matches = sum(1 for pattern in SPAM_PATTERNS if re.search(pattern, text))
    return min(matches * 0.3, 0.8)  # Cap at 0.8

def check_comment_characteristics(comment: CommentInfo) -> float:
    """Check comment characteristics that indicate bots."""
    text = comment.text
    score = 0.0

    # Very short comments
    if len(text.strip()) < 10:
        score += 0.2

    # Too many emojis
    if emoji_count(text) > len(text.split()) * 0.5:
        score += 0.3

    # All caps
    if text.isupper() and len(text) > 10:
        score += 0.2

    # Excessive punctuation
    punct_ratio = len(re.findall(r'[!?]{2,}', text)) / max(len(text), 1)
    if punct_ratio > 0.1:
        score += 0.2

    # No actual words (just numbers/symbols)
    words = re.findall(r'\b[a-zA-Z]{2,}\b', text)
    if len(words) < 2 and len(text) > 5:
        score += 0.4

    return min(score, 0.8)

def check_author_patterns(author: str) -> float:
    """Check author name patterns that indicate bots."""
    score = 0.0

    # Random character patterns
    if re.match(r'^[a-zA-Z]+\d{4,}$', author):  # Name followed by many digits
        score += 0.3

    # Generic names
    generic_patterns = [
        r'(?i)^(user|guest|member|visitor)\d+',
        r'(?i)^[a-z]{6,12}\d{2,4}$',  # Random letters + numbers
    ]

    for pattern in generic_patterns:
        if re.match(pattern, author):
            score += 0.2
            break

    return min(score, 0.5)

def is_bot_comment(comment: CommentInfo) -> Tuple[bool, float]:
    """
    Detect if a comment is likely from a bot.
    Returns (is_bot, bot_score)
    """
    reasons = []

    # Check for spam patterns
    spam_score = check_spam_patterns(comment.text)

    # Check comment characteristics
    char_score = check_comment_characteristics(comment)

    # Check author patterns
    author_score = check_author_patterns(comment.author)

    # Calculate total score
    total_score = spam_score + char_score + author_score

    # Add reasons for debugging
    if spam_score > 0:
        reasons.append(f"spam({spam_score:.2f})")
    if char_score > 0:
        reasons.append(f"chars({char_score:.2f})")
    if author_score > 0:
        reasons.append(f"author({author_score:.2f})")

    # Normalize score (max possible is ~1.5)
    final_score = min(total_score / 1.5, 1.0)
    is_bot = final_score > 0.5

    if is_bot:
        print(f"🤖 Bot detected ({final_score:.2f}): {reasons} - '{comment.text[:50]}...'")

    return is_bot, final_score

def is_english_comment(comment: CommentInfo) -> bool:
    """
    Check if a comment is in English using multiple methods.
    """
    text = comment.text.strip()

    # Skip very short comments or emoji-only comments
    if len(text) < 3:
        return True  # Give benefit of doubt for very short comments

    try:
        detected_lang = detect(text)
        if detected_lang == 'en':
            return True
        return False
    except LangDetectException:
        logger.warning(f"LangDetect failed for comment: '{text[:50]}...'")
        pass

    return False

def identify_duplicate_indices(comments: List[CommentInfo]) -> Set[int]:
    """
    Identify all duplicate comment indices (including first occurrence).
    Returns a set of indices to filter out.
    """
    fingerprint_counts = Counter()
    comment_fingerprints = {}

    for i, comment in enumerate(comments):
        fingerprint = create_comment_fingerprint(comment.text)
        fingerprint_counts[fingerprint] += 1
        if fingerprint not in comment_fingerprints:
            comment_fingerprints[fingerprint] = []
        comment_fingerprints[fingerprint].append(i)

    # Mark duplicates (including first occurrence)
    duplicate_indices = set()
    for fingerprint, count in fingerprint_counts.items():
        if count > 1:  # Duplicate found
            duplicate_indices.update(comment_fingerprints[fingerprint])

    return duplicate_indices

def filter_bots(comments: Iterable[CommentInfo]) -> Generator[CommentInfo, None, None]:
    """Filter out only bot comments, keep duplicates and all languages."""
    for comment in comments:
        if not is_bot_comment(comment)[0]:
            yield comment

def filter_non_english(comments: Iterable[CommentInfo]) -> Generator[CommentInfo, None, None]:
    """Filter out only non-English comments, keep bots and duplicates."""
    for comment in comments:
        if is_english_comment(comment):
            yield comment
        else:
            print(f"🌍 Non-English comment filtered: '{comment.text[:50]}...'")

def filter_all_duplicates(comments: Iterable[CommentInfo]) -> Generator[CommentInfo, None, None]:
    """
    Yield only comments that are not duplicated anywhere else (appear exactly once).
    """
    fingerprints = [create_comment_fingerprint(c.text) for c in comments]
    counts = Counter(fingerprints)
    for comment in comments:
        fingerprint = create_comment_fingerprint(comment.text)
        if counts[fingerprint] == 1:
            yield comment

def sanitise_comments(comments: Iterable[CommentInfo]) -> List[CommentInfo]:
    """
    Sanitise comments using the defined pipeline:
    - Filter out bot comments
    - Filter out non-English comments
    - Filter out all duplicates (keep only unique comments)
    """
    comment_filter_pipeline = compose_filters(
        filter_bots,
        filter_non_english,
        # filter_all_duplicates,
    )
    return list(comment_filter_pipeline(comments))


In [80]:
def get_all_video_comments(video_id: str, max_results: int = 100) -> Generator[CommentInfo, None, None]:
    """Yield all comments as a generator for batch processing."""
    if not YOUTUBE_API_KEY:
        raise ValueError("YouTube API key not configured")

    try:
        youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)

        comments_yielded = 0
        next_page_token = None

        while comments_yielded < max_results:
            batch_size = min(100, max_results - comments_yielded)

            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id,
                maxResults=batch_size,
                pageToken=next_page_token
            )
            response = request.execute()
            for item in response.get("items", []):
                if comments_yielded >= max_results:
                    break

                try:
                    comment_info = _parse_comment_item(item, video_id)

                    # Skip channel owner comments
                    if comment_info.channel_id == comment_info.author_channel_id:
                        continue
                    yield comment_info
                    comments_yielded += 1
                except Exception as e:
                    print(f"Error parsing comment: {e}")
                    continue

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break

    except HttpError as e:
        if e.resp.status == 403:
            print(f"Comments disabled for video {video_id} or quota exceeded")
        else:
            print(f"YouTube API error: {e}")
        return
    except Exception as e:
        print(f"Error getting comments: {e}")
        return

def _parse_comment_item(item: Dict[str, Any], video_id: str) -> CommentInfo:
    """Parse a YouTube API comment item into a CommentInfo object."""
    comment_data = item["snippet"]["topLevelComment"]["snippet"]

    return CommentInfo(
        comment_id=item["snippet"]["topLevelComment"]["id"],
        video_id=video_id,
        text=comment_data["textOriginal"],
        author=comment_data["authorDisplayName"],
        like_count=comment_data.get("likeCount", 0),
        published_at=datetime.fromisoformat(
            comment_data["publishedAt"].replace('Z', '+00:00')
        ),
        updated_at=datetime.fromisoformat(
            comment_data["updatedAt"].replace('Z', '+00:00')
        ),
        reply_count=item["snippet"]["totalReplyCount"],
        channel_id=comment_data.get("channelId", ""),
        author_channel_id=comment_data.get("authorChannelId", {}).get("value")
    )

In [81]:
crypto_videos = list(
    video for _, video in zip(range(3), search_youtube_videos("ETH|Ethereum", max_results=10))
)

if crypto_videos:
    test_video = crypto_videos[1]
    print(f"\nGetting comments for: {test_video.title}")

    # Get all comments
    all_comments = get_all_video_comments(test_video.video_id, max_results=50)
    # print(f"Retrieved {len(all_comments)} total comments")
    # print("all_comments:", all_comments)

    # Filter using functional approach
    filtered_comments = sanitise_comments(all_comments)

    print(f"\n✅ Final human comments: {len(filtered_comments)}")
    for i, comment in enumerate(filtered_comments[:5], 1):
        print(f"\n{i}. {comment.author} (👍 {comment.like_count})")
        print(f"   Comment: {comment.text[:100]}...")


Getting comments for: Eth mood🕺🏽dance mood😂🫣


LangDetect failed for comment: '👌👌👌👌👌👌🥰🥰🥰🥰❤️❤️❤️...'
LangDetect failed for comment: '❤❤❤❤...'
LangDetect failed for comment: '♥️♥️♥️♥️♥️♥️♥️♥️♥️♥️♥️♥️♥️...'
LangDetect failed for comment: '❤❤❤...'
LangDetect failed for comment: '❤👌👌...'
LangDetect failed for comment: '❤❤❤😂...'
LangDetect failed for comment: '❤️❤❤❤...'
LangDetect failed for comment: '❤❤❤❤❤...'
LangDetect failed for comment: '😁😁😁...'


🤖 Bot detected (0.53): ['chars(0.80)'] - '👌👌❤️❤️...'
🌍 Non-English comment filtered: 'എന്റെ സായു  നിന്റെ ഡാൻസ് കണ്ടിട്ട് എച്ചൂ ട്ടന് വരെ...'
🌍 Non-English comment filtered: 'Mom smile so cute ❤❤❤...'
🤖 Bot detected (0.53): ['chars(0.80)'] - 'Mom😍👌🏻...'
🌍 Non-English comment filtered: 'I ❤ u family...'
🤖 Bot detected (0.53): ['chars(0.80)'] - '❤❤❤❤❤❤❤...'
🤖 Bot detected (0.53): ['chars(0.80)'] - '😊😊😊😊❤❤❤...'
🌍 Non-English comment filtered: 'Super super super........'
🌍 Non-English comment filtered: 'Amma supr...'
🤖 Bot detected (0.53): ['chars(0.80)'] - '😂😂😂🥰❤️💞...'
🤖 Bot detected (0.53): ['chars(0.80)'] - '❤👌👌👌😂❤❤...'
🌍 Non-English comment filtered: 'Amma poli ❤🎉...'
🌍 Non-English comment filtered: '👌👌👌👌👌👌🥰🥰🥰🥰❤️❤️❤️...'
🤖 Bot detected (0.53): ['chars(0.80)'] - 'Poli 🎉🎉...'
🌍 Non-English comment filtered: 'അമ്മ 👌👌👌അടിപൊളി 👍👍...'
🌍 Non-English comment filtered: 'എല്ലാവരും സൂപ്പർ മൂഡിൽ ആണല്ലോ ❤️❤️❤️❤️❤️...'
🌍 Non-English comment filtered: '❤❤❤❤...'
🌍 Non-English comment filtered: 'Amme de

In [82]:
non_english_comment = CommentInfo(
    comment_id="12345",
    video_id=test_video.video_id,
    text="Top news: $XA19P to the moon 😊😊😊😊😊😊😊😊😊😊😊😊😊😊😊",
    author="Test User",
    like_count=10,
    published_at=datetime.now(),
    updated_at=datetime.now(),
    reply_count=0,
    channel_id=test_video.channel_id,
    author_channel_id="UC1234567890abcdefg"
)
print("\nTesting non-English comment detection:")
print(f"is_english_comment: {is_english_comment(non_english_comment)}")



Testing non-English comment detection:
is_english_comment: True
